In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory, 
                              CombinedMemory)
from langchain.globals import set_verbose

set_verbose(True)

In [2]:
import getpass
import os
import dotenv

dotenv.load_dotenv()

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")
else:
    print("API key loaded from .env file")

from langchain_cohere import ChatCohere

chat  = ChatCohere(model="command-r-plus")

API key loaded from .env file


In [9]:
import dotenv

dotenv.load_dotenv()
from huggingface_hub import login
login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [30]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [31]:


chat.invoke("What is the capital of India?")

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), 'model': '', 'finish_reason': 'stop'}, id='run-333bf83d-89a5-49b2-87d4-2b95ae976821-0')

In [17]:


TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
{message_buffer_log}

Conversation summary:
{message_summary_log}

Human: {question}
AI:
'''

prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [24]:
chat_buffer_memory = ConversationBufferMemory(memory_key = 'message_buffer_log', 
                                              input_key = 'question',
                                              return_messages = False)

chat_summary_memory = ConversationSummaryMemory(llm = ChatHuggingFace(llm=llm, verbose=True), 
                                                memory_key = 'message_summary_log', 
                                                input_key = 'question',
                                                return_messages = False)

chat_combined_memory = CombinedMemory(memories = [chat_buffer_memory, 
                                                  chat_summary_memory])

In [25]:

# print(chat_combined_memory.load_memory_variables({})['message_summary_log'])
chat_combined_memory.load_memory_variables({})

{'message_buffer_log': '', 'message_summary_log': ''}

In [32]:
chain = LLMChain(llm = chat, 
                 prompt = prompt_template, 
                 memory = chat_combined_memory)

In [34]:
#chain.invoke({'question':'''Can you give me an interesting fact I probably didn't know about?'''})
chain.invoke({'question':'''Can you elaborate a bit more on this fact?'''})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
Human: Can you give me an interesting fact I probably didn't know about?
AI: Sure! Did you know that despite its reputation for being shy, the octopus has a sophisticated sense of humor? Observations have shown that they'll often play tricks on predators and other octopuses in small underwater communities, surprisingly engaging in what some scientists call "theater action.” They also indulge in a bit of light-hearted fun by performing these pranks on their siblings as a way to reduce tension and avoid harm

Conversation summary:
The human asks for an interesting fact. The AI provides a fascinating tidbit about octopuses, sharing that despite being thought of as shy, they actually p

{'question': 'Can you elaborate a bit more on this fact?',
 'message_buffer_log': 'Human: Can you give me an interesting fact I probably didn\'t know about?\nAI: Sure! Did you know that despite its reputation for being shy, the octopus has a sophisticated sense of humor? Observations have shown that they\'ll often play tricks on predators and other octopuses in small underwater communities, surprisingly engaging in what some scientists call "theater action.” They also indulge in a bit of light-hearted fun by performing these pranks on their siblings as a way to reduce tension and avoid harm',
 'message_summary_log': 'The human asks for an interesting fact. The AI provides a fascinating tidbit about octopuses, sharing that despite being thought of as shy, they actually possess a sophisticated sense of humor, engaging in play with pranks both to entertain themselves and to manage conflicts with family members.',
 'text': 'I\'m happy to dive deeper into the fascinating world of octopus hu